# Invoke

In [71]:
import platform
import json

import invoke
from invoke import Program, Collection, task

In [2]:
print(f'- python={platform.python_version()}')
print(f'- invoke={invoke.__version__}')

- python=3.7.3
- invoke=1.3.0


Invoke is a Python (2.7 and 3.4+) task execution tool & library, drawing inspiration from various sources like GNU Make and Ruby's Rake tool.

## Example

In [6]:
@task
def say_hello(ctx):
    print('Bounjour!')

@task
def say_bye(ctx):
    print('Bye!')

In [7]:
ns = Collection('MyTasks')

In [8]:
ns.add_task(say_hello)
ns.add_task(say_bye)

In [9]:
program = Program(name='try', namespace=ns)

In [10]:
program.run("try --list", exit=False)

Subcommands:

  say-bye
  say-hello



In [15]:
program.run("try say-hello", exit=False)

Bounjour!


In [11]:
@task(pre=[say_hello], post=[say_bye])
def say_name(ctx, name):
    print(f'Nice to meet you, {name}.')

In [12]:
ns.add_task(say_name)

In [13]:
program.run("try --list", exit=False)

Subcommands:

  say-bye
  say-hello
  say-name



In [11]:
program.run("try say-name John", exit=False)

Bounjour!
Nice to meet you, John.
Bye!


In [76]:
program.run("try say-name John say-bye", exit=False)

Bounjour!
Nice to meet you, John.
Bye!


In [73]:
@task
def print_config(ctx):
    print(json.dumps(ctx.config._config, indent=4, default=lambda x: '[REMOVED]'))

ns.add_task(print_config)

In [75]:
program.run("try print-config", exit=False)

{
    "run": {
        "warn": false,
        "hide": null,
        "shell": "/bin/bash",
        "pty": false,
        "fallback": true,
        "env": {},
        "replace_env": false,
        "echo": false,
        "encoding": null,
        "out_stream": null,
        "err_stream": null,
        "in_stream": null,
        "watchers": [],
        "echo_stdin": null,
        "dry": false
    },
    "runners": {
        "local": "[REMOVED]"
    },
    "sudo": {
        "prompt": "[sudo] password: ",
        "password": null,
        "user": null
    },
    "tasks": {
        "auto_dash_names": true,
        "collection_name": "tasks",
        "dedupe": true,
        "executor_class": null,
        "search_root": null
    },
    "timeouts": {
        "command": null
    }
}


In [90]:
@task
def print_content(ctx, file):
    result = ctx.run(f'cat {file}', warn=True)
    print(result.exited)
    
ns.add_task(print_content)

In [91]:
program.run("try print-content noexist.md")

cat: noexist.md: No such file or directory


1


In [4]:
@task
def task1(ctx):
    print("------task1------")
    print(ctx.config.run.warn)
    
@task
def task2(ctx):
    print("------task2------")
    print(ctx.config.run.warn)
    
@task(pre=[task1], post=[task2])
def task3(ctx):
    print("------task3------")
    print(ctx.config.run.warn)
#     ctx.run('ls', warn=True)
    ctx.run('ls')
    ctx.config.run.warn = True
    print(ctx.config.run.warn)
    
ns.add_task(task3)

In [19]:
ns.configure({'run':{'warn': True}})

In [5]:
program.run("try task3", exit=False)

------task1------
False
------task3------
False
001-invoke.ipynb
True
------task2------
True


In [21]:
@task
def task_python(ctx):
    ctx.run('python', pty=True)
    
ns.add_task(task_python)

In [22]:
# program.run('try task-python', exit=False)

In [1]:
from subprocess import Popen, PIPE

In [10]:
a = Popen(
#       ". $CONDA_PREFIX/etc/profile.d/conda.sh && conda activate my_test && conda env list",
      "conda activate -h",
      shell=True,
      executable="/bin/bash",
      stdin=PIPE,
      stdout=PIPE,
      stderr=PIPE
     )

In [11]:
print("STDOUT:\n", a.stdout.read().decode('utf8'), sep='')

STDOUT:



In [12]:
print("STDERR:\n", a.stderr.read().decode('utf8'), sep='')

STDERR:

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



